# Notebook: Filter Reviews from Collected HTMLs

## Packages

In [1]:
from langdetect import detect
from bs4 import BeautifulSoup
import pandas as pd
import spacy
import json
import nltk
from nltk.tokenize import sent_tokenize

## Settings

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nils_hellwig/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
%%capture
!python -m spacy download de_core_news_lg

In [4]:
nlp = spacy.load("de_core_news_lg")

## Constants

In [6]:
REVIEWS_PATH = "../datasets/reviews_urls.csv"
RANDOM_STATE = 43

## Code

### Load Dataset

In [7]:
reviews_df = pd.read_csv(REVIEWS_PATH)

### Load Reviews

In [8]:
columns = ['review_id', 'restaurant_id', 'page_index', 'title', 'date', 'author_name', 'author_location', 'text', 'rating', 'restaurant_name', 'language_code']
data_reviews = []

In [9]:
def load_review(review_soup):
    review = {}
    review["title"] = review_soup.find("div", attrs={"class": "quote"}).get_text()
    review["date"] = review_soup.find(class_='ratingDate')['title']
    review["author_name"] = review_soup.find(class_='scrname').get_text()
    user_location_element = review_soup.find(class_='userLocation')
    if user_location_element:
        user_location = user_location_element.get_text()
    else:
        user_location = None
    review["author_location"] = user_location
    review["text"] = review_soup.find(class_='partial_entry').get_text()
    review["rating"] = int(review_soup.find(class_='reviewItemInline').find('span', class_='ui_bubble_rating')['class'][1].split('_')[1]) / 10
    return review

In [10]:
for index, row in reviews_df.iterrows():
    path_review = "../datasets/reviews_restaurants_html/restaurant_" + str(row['restaurant_id']) + "_review_" + str(row["review_id"]) + ".html"
    with open(path_review, 'r', encoding='utf-8') as file:
        html_content = file.read()
    doc_soup = BeautifulSoup(html_content, 'html.parser')
    review_soup = doc_soup.find(id="review_"+str(row["review_id"]))
    review = load_review(review_soup)
    review["restaurant_name"] = doc_soup.find('a', class_='HEADING').get_text()
    review["language_code"] = doc_soup.find("div", class_="prw_reviews_user_links_hsx").span["data-language"]
    review["review_id"] = row["review_id"]
    review["restaurant_id"] = row["restaurant_id"]
    review["page_index"] = row["page_index"]
    data_reviews.append(review)

In [11]:
df_reviews = pd.DataFrame(data_reviews, columns=columns)

### Delete Examples without Data

There are rare cases where the text from the rating is not returned with the GET request to the page from the restaurant rating. These will now be excluded.

In [12]:
df_reviews = df_reviews.drop(df_reviews[(df_reviews['text'] == '') | (df_reviews['title'] == '')].index)

### Filter Languages

We are only considering reviews in german language.

In [13]:
df_reviews = df_reviews.drop(df_reviews[(df_reviews['language_code'] != 'de')].index)

Even if the language code = "de", we have observed that reviews in other languages sometimes have "de" as the language code. These are excluded with the help of Google's language-detection.

In [15]:
df_reviews['detected_language'] = df_reviews['text'].apply(lambda x: detect(x))
# Examples of reviews that were excluded
df_reviews[df_reviews['detected_language'] != 'de']

,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,restaurant_name,language_code,detected_language
17,844399616,715489,1,Scharf aber lecker/très épicé mais délicieux,24. Juni 2022,cestmoit2022,"Karlsruhe, Deutschland","Tolles Essen . Sehr nettes Personal.Scharf wie indisches Essen seien soll. Mit Joghurt dass wir deswegen bekommen haben schön zum Genießen Keine indische Küche mit europäischen Touch. Nur zu empfehlen Un très bon repas. Personnel très accueillant. Très épicé comme doit l être un repas indien. Avec le yaourt que l'on nous a servi pour ça, très appréciable. Pas de la cuisine indienne a l'européene.Super",5.0,Indisches Restaurant Maharadscha,de,fr
185,424688293,715489,18,Es ist immer super دائما رائع,3. Oktober 2016,Shaalan82,None,مكان رائع لتجربة الوجبات الهندية اصحاب المكان يظهرون الود و الترحاب للزوار كانت تجربتي مع المطعم التجربة الثانية و اعتقد انها ستتكرر كلما كانت عندي فرصة لزيارة برلينEin super Restaurant und sehr leckeres Essen und vor allem sehr sehr nette Mitarbeiter.,4.0,Indisches Restaurant Maharadscha,de,ar
250,635997457,8596850,5,Mittagessen!!,27. November 2018,Swati S,None,Essen immer lecker und service immer freundlich und ambiente atmosphere very friendly highly recommended!!!,5.0,Vedis,de,en
339,619684688,8596850,13,Alles sehr gut!,26. September 2018,Nathalia C,None,Das Essen war sehr lecker! The service was great as well as the environment! Super empfehlenswert!!!,5.0,Vedis,de,en


In [16]:
df_reviews = df_reviews[df_reviews['detected_language'] == 'de']

### Check for Duplicates

In [17]:
df_reviews[df_reviews["review_id"] == 904188884]

,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,restaurant_name,language_code,detected_language
800,904188884,13003299,0,Leckeres Essen,19. Juli 2023,Hanna U,None,Das Essen war sehr lecker und ich bin satt geworden. Die Bedingung war sehr gut. Ich komme gerne wieder.,5.0,L'Osteria,de,de
902,904188884,13003299,11,Leckeres Essen,19. Juli 2023,Hanna U,None,Das Essen war sehr lecker und ich bin satt geworden. Die Bedingung war sehr gut. Ich komme gerne wieder.,5.0,L'Osteria,de,de
912,904188884,13003299,12,Leckeres Essen,19. Juli 2023,Hanna U,None,Das Essen war sehr lecker und ich bin satt geworden. Die Bedingung war sehr gut. Ich komme gerne wieder.,5.0,L'Osteria,de,de
922,904188884,13003299,13,Leckeres Essen,19. Juli 2023,Hanna U,None,Das Essen war sehr lecker und ich bin satt geworden. Die Bedingung war sehr gut. Ich komme gerne wieder.,5.0,L'Osteria,de,de
932,904188884,13003299,14,Leckeres Essen,19. Juli 2023,Hanna U,None,Das Essen war sehr lecker und ich bin satt geworden. Die Bedingung war sehr gut. Ich komme gerne wieder.,5.0,L'Osteria,de,de
942,904188884,13003299,15,Leckeres Essen,19. Juli 2023,Hanna U,None,Das Essen war sehr lecker und ich bin satt geworden. Die Bedingung war sehr gut. Ich komme gerne wieder.,5.0,L'Osteria,de,de
952,904188884,13003299,16,Leckeres Essen,19. Juli 2023,Hanna U,None,Das Essen war sehr lecker und ich bin satt geworden. Die Bedingung war sehr gut. Ich komme gerne wieder.,5.0,L'Osteria,de,de
962,904188884,13003299,17,Leckeres Essen,19. Juli 2023,Hanna U,None,Das Essen war sehr lecker und ich bin satt geworden. Die Bedingung war sehr gut. Ich komme gerne wieder.,5.0,L'Osteria,de,de
972,904188884,13003299,18,Leckeres Essen,19. Juli 2023,Hanna U,None,Das Essen war sehr lecker und ich bin satt geworden. Die Bedingung war sehr gut. Ich komme gerne wieder.,5.0,L'Osteria,de,de
982,904188884,13003299,19,Leckeres Essen,19. Juli 2023,Hanna U,None,Das Essen war sehr lecker und ich bin satt geworden. Die Bedingung war sehr gut. Ich komme gerne wieder.,5.0,L'Osteria,de,de


In [18]:
duplicate_rows = df_reviews[df_reviews.duplicated(subset=['review_id'], keep=False)]
duplicate_rows

,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,restaurant_name,language_code,detected_language
800,904188884,13003299,0,Leckeres Essen,19. Juli 2023,Hanna U,None,Das Essen war sehr lecker und ich bin satt geworden. Die Bedingung war sehr gut. Ich komme gerne wieder.,5.0,L'Osteria,de,de
801,904187385,13003299,0,Schönes Restaurant,19. Juli 2023,Franka P,None,Mir hat das Essen sehr gut geschmeckt und die Preise waren angemessen. Die Bedienung war freundlich.,5.0,L'Osteria,de,de
802,901181174,13003299,0,Mieser Service usw.,7. Juli 2023,M4DM4N,"Gießen, Deutschland","Wir haben obwohl kaum etwas los war erstmal 10-15 Minuten gewartet bis wir bestellen durften. Die Bestellung der Rechnung hat dann am Ende nochmal genauso lange gedauert. Hatte eine Pizza Canaria. Die hätte echt gut sein können, wenn nicht die Sardellen komplett gefehlt hätten und Salami, Schinken und Champignons auf getrennten Bereichen gelegen hätten ala Pizza 4-Jahreszeiten.Zu letzterem hätte eine Warnung auf der Karte oder von der Bedienung gereicht. Zusätzlich waren die Teller viel zu klein für die Pizzen, wodurch man bis zur Hälfte erstmal den Tisch mit der Pizza gewischt hat.",3.0,L'Osteria,de,de
803,882814591,13003299,0,Gute italienische Küche nach Hausmacher Art,19. März 2023,NGBerlin,"Berlin, Deutschland","Haben heute am Humboldthafen die L'Osteria besucht und waren sehr angenehm und positiv überrascht, wie lecker das Essen hier ist. Die Preise sind angemessen, die Portionen für den großen Hunger bemessen. Der Service ist freundlich und sehr nett",4.0,L'Osteria,de,de
804,880599815,13003299,0,Nichts besonderes!,1. März 2023,dreamteambln,"Berlin, Deutschland","Das Nudelgericht, Spaghetti mit Ragout in Rotweinsoße hörte sich spannend an. Leider musste ich knapp 20 Minuten warten, bis ich bestellen konnte. Das Essen war sowas von nach nichts schmeckend und langweilig, dass ich kräftig nachwürzen musste. Das Ragout in Rotweinsoße schmeckte wie eine ganz normale Spaghetti Bolognese. Ich als Berliner würde hier kein weiteres Mal essen gehen.",3.0,L'Osteria,de,de
...,...,...,...,...,...,...,...,...,...,...,...,...
987,873905925,13003299,19,Beste Pizza 🍕,4. Januar 2023,Michl K,None,"Sehr gute und riesige Pizza 🍕 Personal sehr freundlich und zuvorkommend. Essen wird auch sehr schnell serviert, obwohl viel los war😊",5.0,L'Osteria,de,de
988,843970416,13003299,19,Weniger ist mehr,22. Juni 2022,elkef565,None,"Ich habe die Restaurantkette schon in unterschiedlichen Städten besucht. Auch in Berlin an verschiedenen Standorten. Die Atmosphäre ist angenehm locker, der Service immer freundlich und die Qualität sowie Preis-Leistungsverhältnis sehr gut. Mein einziger Kritikpunkt ist die schon unschöne Größe der Pizza, die den Tellerrand überlappt und fast von niemandem aufgegessen werden kann. Es ärgert mich zu sehen , was da an Resten auf den Tellern bleibt und in den Müll wandert. Hier wäre tatsächlich wenig mehr.",4.0,L'Osteria,de,de
989,841705003,13003299,19,Sehr lecker und üppig!,6. Juni 2022,756alexxh,"Weingarten, Deutschland","Das Essen war ausgezeichnet und die Bedienung sehr nett! Wir hatten Pasta und Desserts, alles war super! Wir kommen wieder!",5.0,L'Osteria,de,de
990,834739169,13003299,19,Ausgezeichnet,16. April 2022,thibaultwwww,None,Das Personal ist sehr witzig und der Ort ist warm. Die Pizza die ich gegessen habe war köstlich und sehr gross !,5.0,L'Osteria,de,de


### Anonymisierung

In [19]:
df_reviews["text_noanonymization"] = df_reviews["text"]

In [20]:
def anonymize_entities(text):
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ in ["LOC", "PERSON", "DATE"] and ent.label_ != "Essen":
            text = text.replace(ent.text, f"<{ent.label_}>")
    return text

df_reviews["text"] = df_reviews["text"].apply(anonymize_entities)

In [21]:
def anonymize_username(text, username):
    return text.replace(username, "<PERSON>")
df_reviews["text"] = df_reviews.apply(lambda row: anonymize_username(row["text"], row["author_name"]), axis=1)

### Filter Sentences

In [22]:
df_reviews_sentences = pd.DataFrame(columns=list(df_reviews.columns) + ['sentence_idx'])

for idx, row in df_reviews.iterrows():
    sentences = sent_tokenize(row['text'])
    sentence_index = 0
    for sentence in sentences:
        new_row = row.copy()  # Kopiere die gesamte Zeile
        new_row['text'] = sentence  # Setze 'text' auf den aktuellen Satz
        new_row['sentence_idx'] = idx
        df_reviews_sentences.loc[len(df_reviews_sentences)] = new_row
        sentence_index += 1

### Randomisierung

In [23]:
df_reviews_sentences = df_reviews_sentences.sample(frac=1, random_state=RANDOM_STATE)  # 'frac=1' mischt den gesamten DataFrame
df_reviews_sentences.reset_index(drop=True, inplace=True)
df_reviews_sentences

,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,restaurant_name,language_code,detected_language,text_noanonymization,sentence_idx
0,843970416,13003299,12,Weniger ist mehr,22. Juni 2022,elkef565,None,"Es ärgert mich zu sehen , was da an Resten auf den Tellern bleibt und in den Müll wandert.",4.0,L'Osteria,de,de,"Ich habe die Restaurantkette schon in unterschiedlichen Städten besucht. Auch in Berlin an verschiedenen Standorten. Die Atmosphäre ist angenehm locker, der Service immer freundlich und die Qualität sowie Preis-Leistungsverhältnis sehr gut. Mein einziger Kritikpunkt ist die schon unschöne Größe der Pizza, die den Tellerrand überlappt und fast von niemandem aufgegessen werden kann. Es ärgert mich zu sehen , was da an Resten auf den Tellern bleibt und in den Müll wandert. Hier wäre tatsächlich wenig mehr.",918
1,618244459,8596850,16,Nettes Treffen,20. September 2018,Marvin G,None,Wohlfühlambiente mit toller Bedienung!Die Cocktails waren sehr zu empfehlen und in der Happy Hour!,5.0,Vedis,de,de,Sehr gutes Essen! Wohlfühlambiente mit toller Bedienung!Die Cocktails waren sehr zu empfehlen und in der Happy Hour!,361
2,324877682,944596,14,Sehr uriges Berliner Original mit echt leckerem Essen,6. November 2015,ThomasQu,"Wiesbaden, Deutschland",Ich war sicher nicht das letzte Mal da...,5.0,Hackethals,de,de,"Das Menü steht auf einer Tafel und wird demzufolge tagesaktuell geändert. Das Essen war sehr lecker, die Atmosphäre urgemütlich und entspannt und der Wirt wie auch der Rest des Teams sehr nett. Das Hausbier ist sehr zu empfehlen und das Publikum - nicht zuletzt wegen TripAdvisor - sehr international. Ich war sicher nicht das letzte Mal da...",548
3,258655908,944596,17,Nicht toll,9. März 2015,xinos1991,None,Ich weiss nicht ob wir nur eine schlechte tag erwischt haben !,1.0,Hackethals,de,de,Ich weiss nicht ob wir nur eine schlechte tag erwischt haben ! EsWar recht voll aber trotzden konnte alles schneller und mit mehr freundlichkeit sein ! Das essen hat nicht besondre geschmeckt und ich denke das wir nicht nochmal zum essen hin gehen !,570
4,595896509,778662,4,Personal extrem überfordert,13. Juli 2018,Magdalene,"Berlin, Deutschland","Personal extrem überfordert, ohne viel Publikum.",1.0,Vapiano,de,de,"Personal extrem überfordert, ohne viel Publikum. Wartezeit für Salat 40 min. Kleidung der Köche sehr verschmutzt. Zutaten werden ohne Handschuhe angefasst. Brot wurde gleich ganz weggelassen. Beim Gehen, die Frage, ob alles zur Zufriedenheit war mit - nein - beantwortet, trotzdem keine Nachfrage oder Entschuldigung erhalten.Die Individualität ist leider dem Ganz Gewöhnlichem gewichen. Der Italiener um die Ecke kann es besser !",640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4022,636243658,778662,2,"Tolles Ambiente, glutenfrei kein Problem",28. November 2018,703uschig,None,"Zum Bestellen muss man sich an den verschiedenen Stationen anstellen, das ist etwas nervig, besonders wenn man Kinder dabei hat.",4.0,Vapiano,de,de,"Wir wurden freundlich begrüßt und uns wurde die Vorgehensweise erklärt. Wir fanden einen schönen Platz in dem riesigen, hohen Raum mit guter Luft. Zum Bestellen muss man sich an den verschiedenen Stationen anstellen, das ist etwas nervig, besonders wenn man Kinder dabei hat. Glutenfrei ist auf der Karte gekennzeichnet und ich konnte auch direkt mit dem Pizzabäcker und der Salatbereiterin darüber sprechen. Das war beruhigend. Geschmacklich war alles o.k., preislich eben Berliner Niveau. Wir würden wieder hingehen.",623
4023,904188884,13003299,16,Leckeres Essen,19. Juli 2023,Hanna U,None,Ich komme gerne wieder.,5.0,L'Osteria,de,de,Das Essen war sehr lecker und ich bin satt geworden. Die Bedingung war sehr gut. Ich komme gerne wieder.,952
4024,643947656,778662,2,"sehr leckere Pizza, Pasta & Dessert",6. Januar 2019,Nela311,"Kiel, Deutschland","Daraufhin war unsere Pizza und Pasta gleichzeitig fertig - wir konnten also schön gemeinsam Essen, was im Vapiano ja nicht im

### Balancing

In [50]:
df_reviews_sentences[df_reviews_sentences['restaurant_id'] == 944596]['rating'].value_counts()

rating
5.0    622
4.0     68
1.0     49
3.0     21
2.0      8
Name: count, dtype: int64

In [35]:
df_reviews_sentences['restaurant_id'].value_counts()

restaurant_id
778662      999
13003299    831
715489      805
944596      768
8596850     624
Name: count, dtype: int64

In [49]:
grouped = df_reviews_sentences.groupby(['rating', 'restaurant_id'], group_keys=False).apply(lambda x: x.sample(8, replace=False, random_state=RANDOM_STATE))
grouped.reset_index(drop=True, inplace=True)
grouped

,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,restaurant_name,language_code,detected_language,text_noanonymization,sentence_idx
0,522918729,715489,13,Über eine Stunde aufs Essen gewartet,9. September 2017,PascalT648,"Bern, Schweiz",Schlechter Service.,1.0,Indisches Restaurant Maharadscha,de,de,Schlechter Service. Schon nur das Bestellen hat locker 20 Minuten gedauert. Das Essen kam dann so 80 Minuten später. Niemehr,134
1,522918729,715489,13,Über eine Stunde aufs Essen gewartet,9. September 2017,PascalT648,"Bern, Schweiz",Niemehr,1.0,Indisches Restaurant Maharadscha,de,de,Schlechter Service. Schon nur das Bestellen hat locker 20 Minuten gedauert. Das Essen kam dann so 80 Minuten später. Niemehr,134
2,558216768,715489,11,Völlig überwertet,4. Februar 2018,587christiane,None,Toiletten dürften ruhig mal saniert werden.2.,1.0,Indisches Restaurant Maharadscha,de,de,"1. Lokal: ist schlicht, funktional und nicht zu kitschig eingerichtet. Toiletten dürften ruhig mal saniert werden.2. Essen: überraschend schlecht, angefangen von Vorspeisen (in Kichererbsenteig panierte Fleischvariationen) bis zum Hauptgang wirklich mau. Soßen erinnern geschmacklich an Fertigprodukte, Fleischqualität nicht die Beste, alles sehr geschmacklos und beliebig, einfach nur enttäuschend von indischer Geschmacksexplosion / tollen Gewürzen nichts zu schmecken 3. Service: ein kleiner Lichtblick, nett, freundlich und aufmerksam!",118
3,767954111,715489,5,'Garnelen' aus Surimi,30. August 2020,Pfingstr0se,"Eresing, Deutschland",Die Garnelen waren tatsächlich aus Pressfisch.,1.0,Indisches Restaurant Maharadscha,de,de,Das sagt schon alles. Die Garnelen waren tatsächlich aus Pressfisch. Das konnte auch die primitive Tomatensauce nicht übertünchen. Und teuer waren die wenigen Stücke auch noch. Nie wieder gehe ich da rein.,58
4,767954111,715489,5,'Garnelen' aus Surimi,30. August 2020,Pfingstr0se,"Eresing, Deutschland",Das konnte auch die primitive Tomatensauce nicht übertünchen.,1.0,Indisches Restaurant Maharadscha,de,de,Das sagt schon alles. Die Garnelen waren tatsächlich aus Pressfisch. Das konnte auch die primitive Tomatensauce nicht übertünchen. Und teuer waren die wenigen Stücke auch noch. Nie wieder gehe ich da rein.,58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,764836304,13003299,3,Super Restaurant,14. August 2020,thesi-st,None,Ich war hier mit einer Freundin zu zweit spontan essen.,5.0,L'Osteria,de,de,Ich war hier mit einer Freundin zu zweit spontan essen. Die Bedienung hat sicher sofort bemüht und noch einen freien Tisch zu finden. Super lieb und auch das Essen war sehr lecker. Auf jeden Fall einen Besuch wert ☺️,832
196,775855576,13003299,2,War lecker !,28. Oktober 2020,lesannel,None,Die Pizza ist super und das Ambiente ist auch top.,5.0,L'Osteria,de,de,War sehr lecker und die Leute hier sind alle prima! Die Pizza ist super und das Ambiente ist auch top.,820
197,615588680,13003299,9,Super Pizza und nettes Personal,11. September 2018,lauralola1234,None,"Wir kamen am Freitagabend spontan her, es war natürlich sehr viel los.",5.0,L'Osteria,de,de,"Wir kamen am Freitagabend spontan her, es war natürlich sehr viel los. Wir mussten schon ein bisschen warten, aber die Bedienung war sehr bemüht, dass wir schnell einen Platz bekamen. Sie war wirklich sehr nett.Es war wirklich stressig, trotzdem war das gesamte Personal total freundlich. Die Pizzen sind riesig, es reicht also, wenn man sich eine teilt. Man kann auch 2 unterschiedliche Hälften bestellen. Sie war wirklich sehr, sehr lecker. Auch der gemischte Salat und der Aperol Spritz waren sehr gut. Würde auf jeden Fall gerne wieder herkommen!",891
198,764048286,13003299,3,Top Italienische Küche,10. August 2020,H2525OXleal,None,"Sehr schönes Italienisches Restaurant, top Preis-Leistungs-Verhältnis, sehr nette Bedienung, gut organisiert.",5.0,L'Osteria,de,de,"Sehr schönes Italienisches Restaurant, top Preis-Leistungs-Verhältnis, sehr nette Bedienung, gut organisi

### Store as .csv 

In [25]:
df_reviews.to_csv("../datasets/reviews.csv")

In [26]:
df_reviews_sentences.to_csv("../datasets/reviews_sentences.csv")